# Guessing Bits: Improved Lattice Attacks on(EC)DSA with Nonce Leakage (Sun et al., 2021) LeetArxiv Implementation

This coding guide is best followed alongside this [LeetArxiv article link](https://leetarxiv.substack.com/p/guessing-bits-improved-lattice-attacks).



The paper, *Guessing Bits: Improved Lattice Attacks on(EC)DSA with Nonce Leakage* , improves on the (Albrecht & Heninger, 2020) lattice-based HNP attack by:


1. Guessing some secret key bits to increase attack success probability.

2. Decomposing the secret key into batches to recover parts of the secret. ie. it’s no longer an ‘all-or-nothing’ approach.

## Install Libraries
We need:
1. **fpylll** for lattice construction.
2. **cysignals** to interact with fpylll's C++ code

In [1]:
#Install fpylll for lattices
!pip install fpylll
!pip install cysignals

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.6/41.6 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.7/270.7 kB 10.7 MB/s eta 0:00:00


## Recentering and Projected Lattice

This section constructs a lattice with both the 'recentering trick' and 'projection trick' as described in section 2.5 and 2.6 of (Sun et al., 2021) [here](https://leetarxiv.substack.com/p/guessing-bits-improved-lattice-attacks)

We recover the secret key $\alpha$ from the second last row element using the equation:

$\alpha = t_0^{-1} \cdot (a_0 \cdot (\text{target} / (2^{l+1})) \pmod{q}$

In [2]:
import math
import random
from random import randint
from fpylll import FPLLL
from fpylll import *
from fpylll.util import gaussian_heuristic
from decimal import Decimal
from time import time

#Function to eliminate secret key
def EliminateSecretKeyTransform(prime, multipliers, observations):
  if len(multipliers) != len(observations):
    raise ValueError(f'Expected number of observations to equal number of multipliers, but got {len(observations)} and {len(multipliers)}.')
  m = len(multipliers)
  t0_inv = pow(multipliers[0], -1, prime)
  multipliers_new = []
  observations_new = []

  for i in range(1, m):
    ti_prime = (multipliers[i] * t0_inv) % prime
    ai_prime = (observations[i] - ti_prime * observations[0]) % prime
    multipliers_new.append(ti_prime)
    observations_new.append(ai_prime)

  return multipliers_new, observations_new, multipliers[0], observations[0]

"""
prime = q
multipliers = t
observations = v
noOfLeakedBits = l
latticeDimension = d
"""
def HNP_EliminateSecretKey_Projection(prime, multipliers, observations, errorBound, noOfLeakedBits):
  latticeDimension = len(multipliers) + 1
  #Eliminate the secret key
  multipliers_new, observations_new, t0, a0 = EliminateSecretKeyTransform(prime, multipliers, observations)
  assert(latticeDimension == len(multipliers_new)+2)
  #Build Projected lattice
  lattice = IntegerMatrix(latticeDimension, latticeDimension)
  #Set identity to prime * 2^(noOfLeakedBits + 1)
  for i in range(len(multipliers_new)):
    lattice[i, i] = int(prime) * (2 ** (noOfLeakedBits + 1))
  #Stack (multipliers and observations) * 2^(noOfLeakedBits + 1)
  for j in range(len(multipliers_new)):
    lattice[len(multipliers_new), j] = multipliers_new[j] * (2 ** (noOfLeakedBits + 1))
    lattice[len(multipliers_new)+1, j] = observations_new[j] * (2 ** (noOfLeakedBits + 1))
  #We replace B/p with 1 * (2 ** (noOfLeakedBits + 1)
  lattice[len(multipliers_new), len(multipliers_new)] = int(1) * (2 ** (noOfLeakedBits + 1))
  #Set error bound
  lattice[len(multipliers_new)+1, len(multipliers_new)+1] = errorBound
  #Perform LLL reduction
  #print(lattice)
  LLL.reduction(lattice)
  t0_inv = pow(t0, -1, prime)
  leakInverse = pow(2 ** (noOfLeakedBits + 1), -1, prime)
  #Search in lattice rows for secret key
  secretKey = prime
  for i in range(lattice.nrows):
    row = [lattice[i, j] for j in range(lattice.ncols)]
    #Target changes
    targetValue = row[-2] // (2 ** (noOfLeakedBits + 1))
    if row[-1] == -errorBound:
      alpha = (t0_inv * (a0 + targetValue)) % prime
      print(f"{alpha} : {math.log(alpha,2)}")
      if(alpha < secretKey):
        secretKey = alpha
    if row[-1] == errorBound:
      alpha = (t0_inv * (a0 - targetValue)) % prime
      print(f"{alpha} : {math.log(alpha,2)}")
      if(alpha < secretKey):
        secretKey = alpha
  return secretKey



In [3]:
def TestHNP_EliminateSecretKey(secretKey):
    #random.seed(9665096)
    _ = FPLLL.set_precision(1000)
    q = 205115282021455665897114700593932402728804164701536103180137503955397371
    equationsToGenerate = 65
    errorMaxExponent = 200
    multiplierMaxExponent = 230
    errorBound = 2**errorMaxExponent
    noOfLeakedBits = math.ceil(math.log(q, 2)) - errorMaxExponent
    multiplierBound= 2**multiplierMaxExponent
    # Generate Dataset
    randomMultipliers = [];results = [];noiseValues = []
    for i in range(equationsToGenerate):
        r = randint(1, multiplierBound);s = randint(0, errorBound)
        c = (r*secretKey + s) % q
        #print(i, " : ", math.log(abs(s - halfErrorBound), 2))
        randomMultipliers.append(r);results.append(c);noiseValues.append(s)

    #Predict
    prediction = HNP_EliminateSecretKey_Projection(q, randomMultipliers, results, errorBound,noOfLeakedBits)
    print(f"Target: {secretKey}")
    print(f"Prediction: {prediction}: {math.log(prediction,2)}")

if __name__ == "__main__":
  randomValue = (randint(2**134, 2**135))
  secretKey = randomValue
  TestHNP_EliminateSecretKey(secretKey)

27189065067611292769837543284525334747589 : 134.3201503387104
Target: 27189065067611292769837543284525334747589
Prediction: 27189065067611292769837543284525334747589: 134.3201503387104


## Increasing Lattice Volume

Here’s the paper’s thesis: we don’t need to find the entire secret key at once, we can split it and search in batches.

More formally, we can modify the lattice to increase its volume while the error vector remains constant.

This is Section 3.0 of the [LeetArxiv article](https://leetarxiv.substack.com/p/guessing-bits-improved-lattice-attacks).

There are two ways to increase lattice volume:
1. Multiply the observations by a constant factor.
2. Multiply the secret key by a constant factor.

### Increasing Volume via Multiplying Observations by a constant

In [13]:
def HNP_BuildLattice_GuessingBits(prime, multipliers, observations, noOfLeakedBits, noOfGuessedBits, lsb, embeddingFactor):
  d = len(multipliers)
  lattice = IntegerMatrix(d+2, d+2)
  #Set prime identity
  for i in range(d):
    lattice[i, i] = int(prime) * (2 ** (noOfLeakedBits + 1))

  #Set observation and multiplier rows
  for j in range(d):
    lattice[d, j] = multipliers[j] * (2 ** (noOfLeakedBits + 1 + noOfGuessedBits))
    lattice[d+1, j] = (observations[j] - lsb * multipliers[j]) * (2 ** (noOfLeakedBits + 1))
  lattice[d, d] = 2 ** noOfGuessedBits
  lattice[d+1, d+1] = embeddingFactor
  return lattice

def HNP_GuessingBits(prime, multipliers, observations, errorBound, noOfLeakedBits, noOfGuessedBits):
  d = len(multipliers)
  embeddingFactor = errorBound
  bkzBlockSize = 20
  param = BKZ.Param(block_size=bkzBlockSize, flags=BKZ.AUTO_ABORT)
  for lsb in range(0, 2 ** noOfGuessedBits):
    lattice = HNP_BuildLattice_GuessingBits(prime, multipliers, observations, noOfLeakedBits, noOfGuessedBits, lsb,embeddingFactor)
    #BKZ.reduction(lattice,param)
    LLL.reduction(lattice)
    for i in range(d+2):
      temp = abs(lattice[i, d])
      alpha0 = ((temp >> noOfGuessedBits) << noOfGuessedBits) + lsb
      alpha1 = prime - ((temp >> noOfGuessedBits) << noOfGuessedBits)+lsb
      logAlpha0 = 0; logAlpha1=0;
      if(alpha0 > 0):
        logAlpha0 = math.log(alpha0,2)
        if(logAlpha0 < 140):
          print(f"alpha0: {alpha0}, {logAlpha0}")
      if(alpha1 > 0):
        logAlpha1 = math.log(alpha1,2)
        if(logAlpha1 < 140):
          print(f"alpha1: {alpha1}, {logAlpha1}")
    break #Remove this to enumerate the entire set

    print(f"Done lsb:{lsb}\n")


def TestHNP_GuessingBits(secretKey, noOfGuessedBits):
    _ = FPLLL.set_precision(1000)
    q = 205115282021455665897114700593932402728804164701536103180137503955397371
    equationsToGenerate = 44
    errorMaxExponent = 200
    multiplierMaxExponent = 95
    errorBound = 2**errorMaxExponent
    noOfLeakedBits = math.ceil(math.log(q, 2)) - errorMaxExponent
    halfErrorBound = errorBound // 2
    # Generate Dataset
    randomMultipliers = [];results = [];noiseValues = []
    for i in range(equationsToGenerate):
        r = randint(0, 2**multiplierMaxExponent)
        s = randint(0, errorBound)
        c = (r*secretKey + s) % q
        c = (c - halfErrorBound) % q
        randomMultipliers.append(r);results.append(c);noiseValues.append(s)
    print(f"Secret key: {secretKey}, {math.log(secretKey,2)}")
    HNP_GuessingBits(q, randomMultipliers, results, errorBound//2,noOfLeakedBits, noOfGuessedBits)

if __name__ == "__main__":
  #random.seed(96650956)
  randomValue = (randint(2**134, 2**135))
  noOfGuessedBits = 100
  secretKey = randomValue
  TestHNP_GuessingBits(secretKey, noOfGuessedBits)


Secret key: 31687962778238317991465423013831615722838, 134.54105870721187
alpha0: 1267650600228229401496703205376, 100.0
alpha0: 31687962777159791100054910624046698725376, 134.54105870716276


### Increasing volume via secret key multiplication

In [5]:
import math
import random
from random import randint
from fpylll import FPLLL
from fpylll import *
from fpylll.util import gaussian_heuristic
from decimal import Decimal
from time import time

#Function to eliminate secret key
def EliminateSecretKeyTransform_Embedding(prime, multipliers, observations, embeddingFactor):
  if len(multipliers) != len(observations):
    raise ValueError(f'Expected number of observations to equal number of multipliers, but got {len(observations)} and {len(multipliers)}.')
  m = len(multipliers)
  t0_inv = pow(multipliers[0], -1, prime)
  multipliers_new = []
  observations_new = []

  for i in range(1, m):
    ti_prime = (multipliers[i] * embeddingFactor * t0_inv) % prime
    ai_prime = (observations[i] - ti_prime * observations[0]) % prime
    multipliers_new.append(ti_prime)
    observations_new.append(ai_prime)

  return multipliers_new, observations_new, multipliers[0], observations[0]

"""
prime = q
multipliers = t
observations = v
noOfLeakedBits = l
latticeDimension = d
"""
def HNP_EliminateSecretKey_Volume(prime, multipliers, observations, errorBound, noOfLeakedBits, volume):
  latticeDimension = len(multipliers) + 1
  #Eliminate the secret key
  multipliers_new, observations_new, t0, a0 = EliminateSecretKeyTransform(prime, multipliers, observations)
  assert(latticeDimension == len(multipliers_new)+2)
  #Build Projected lattice
  lattice = IntegerMatrix(latticeDimension, latticeDimension)
  #Set identity to prime * 2^(noOfLeakedBits + 1)
  for i in range(len(multipliers_new)):
    lattice[i, i] = int(prime) * (2 ** (noOfLeakedBits + 1))
  #Stack (multipliers and observations) * 2^(noOfLeakedBits + 1)
  for j in range(len(multipliers_new)):
    lattice[len(multipliers_new), j] = multipliers_new[j] * (2 ** (noOfLeakedBits + 1))
    lattice[len(multipliers_new)+1, j] = observations_new[j] * (2 ** (noOfLeakedBits + 1))
  #We replace B/p with 1 * (2 ** (noOfLeakedBits + 1 + volume)
  lattice[len(multipliers_new), len(multipliers_new)] = int(1) * (2 ** (noOfLeakedBits + 1 + volume) )
  #Set error bound
  lattice[len(multipliers_new)+1, len(multipliers_new)+1] = errorBound
  #Perform LLL reduction
  #print(lattice)
  LLL.reduction(lattice)
  t0_inv = pow(t0, -1, prime)
  leakInverse = pow(2 ** (noOfLeakedBits + 1), -1, prime)
  #Search in lattice rows for secret key
  secretKey = prime
  for i in range(lattice.nrows):
    row = [lattice[i, j] for j in range(lattice.ncols)]
    #Target changes
    targetValue = row[-2] // (2 ** (noOfLeakedBits + 1 + volume))
    if row[-1] == -errorBound:
      alpha = (t0_inv * (a0 + targetValue)) % prime
      alpha >>= volume
      print(f"{alpha} : {math.log(alpha,2)}")
      if(alpha < secretKey):
        secretKey = alpha
    if row[-1] == errorBound:
      alpha = (t0_inv * (a0 - targetValue)) % prime
      alpha >>= volume
      print(f"{alpha} : {math.log(alpha,2)}")
      if(alpha < secretKey):
        secretKey = alpha
  return secretKey




In [6]:
def TestHNP_EliminateSecretKey(secretKey, volume):

    _ = FPLLL.set_precision(1000)
    q = 205115282021455665897114700593932402728804164701536103180137503955397371
    equationsToGenerate =200
    errorMaxExponent = 233
    multiplierMaxExponent = 125
    errorBound = 2**errorMaxExponent
    noOfLeakedBits = math.ceil(math.log(q, 2)) - errorMaxExponent
    multiplierBound= 2**multiplierMaxExponent
    # Generate Dataset
    randomMultipliers = [];results = [];noiseValues = []
    for i in range(equationsToGenerate):
        r = randint(1, multiplierBound);s = randint(0, errorBound)
        c = (r*secretKey + s) % q
        #print(i, " : ", math.log(abs(s - halfErrorBound), 2))
        randomMultipliers.append(r);results.append(c);noiseValues.append(s)

    #Predict
    prediction = HNP_EliminateSecretKey_Volume(q, randomMultipliers, results, errorBound,noOfLeakedBits,volume)
    secretKey >>= volume
    print(f"Target: {secretKey}\n")
    print(f"Prediction: {prediction}: {math.log(prediction,2)}")

if __name__ == "__main__":
  random.seed(966508)
  randomValue = (randint(2**134, 2**135))
  volume = 100
  secretKey = randomValue << volume

  TestHNP_EliminateSecretKey(secretKey, volume)

35555136710425978440849335508674701825633 : 134.7071817989767
125142003583487477290534086958546341249643 : 136.5226179979561
64635119576618207962251063354638951551476 : 135.569442063689
109754585990801268343934655562937432698765 : 136.3333331130396
26725448859760884990338081943716185051639 : 134.2953379740226
24455342603754846888228127795226289576664 : 134.1672734715493
136119191219376580177292857294413016453759 : 136.6439223747488
36912720599033670937059335695346510650074 : 134.76124186876177
22178994402017465419862453323955942440441 : 134.02631775052924
109304095029702112398758843449729376823312 : 136.32739934233746
36876469719275662541226523254088298407842 : 134.7598243449094
18430665876522343314867103518693024734735 : 133.75923199014204
71370283906269183703604307041540266972223 : 135.7124473069356
37979945093139330364150749661431052485704 : 134.80236161530755
Target: 35555136710425978440849335508674701825533

Prediction: 18430665876522343314867103518693024734735: 133.75923199014204

# Bonus Content

## Double Recenter trick
1. Idk why but you can combine the 'increase exponent by 1' and half the error by 2 trick. So you effectively lower your bound by 2 bits lol

So this works lol

In [7]:
def TestHNP_EliminateSecretKey(secretKey, volume):

    _ = FPLLL.set_precision(1000)
    q = 205115282021455665897114700593932402728804164701536103180137503955397371
    equationsToGenerate =20
    errorMaxExponent = 235
    multiplierMaxExponent = 15
    errorBound = 2**errorMaxExponent
    noOfLeakedBits = 2#math.ceil(math.log(q, 2)) - errorMaxExponent
    multiplierBound= 2**multiplierMaxExponent
    halfErrorBound = errorBound // 2
    # Generate Dataset
    randomMultipliers = [];results = [];noiseValues = []
    for i in range(equationsToGenerate):
        r = randint(1, multiplierBound);s = randint(0, errorBound)
        c = (r*secretKey + s) % q
        c = (c - halfErrorBound) % q
        #print(i, " : ", math.log(abs(s - halfErrorBound), 2))
        randomMultipliers.append(r);results.append(c);noiseValues.append(s)

    #Predict
    prediction = HNP_EliminateSecretKey_Volume(q, randomMultipliers, results, errorBound//2,noOfLeakedBits,volume)
    secretKey >>= volume
    print(f"Target: {secretKey}\n")
    print(f"Prediction: {prediction}: {math.log(prediction,2)}")

if __name__ == "__main__":
  #random.seed(966508)
  randomValue = (randint(2**134, 2**135))
  volume = 100
  secretKey = randomValue << volume

  TestHNP_EliminateSecretKey(secretKey, volume)

94513299310170309808615989957776359119879 : 136.11764114597997
Target: 38215988048184170700738239720338574004399

Prediction: 94513299310170309808615989957776359119879: 136.11764114597997


In [8]:
def TestHNP_EliminateSecretKey(secretKey):
    _ = FPLLL.set_precision(1000)
    q = 205115282021455665897114700593932402728804164701536103180137503955397371
    equationsToGenerate = 250
    errorMaxExponent = 234
    multiplierMaxExponent = 105
    errorBound = 2**errorMaxExponent
    noOfLeakedBits = math.ceil(math.log(q, 2)) - errorMaxExponent
    multiplierBound= 2**multiplierMaxExponent
    halfErrorBound = errorBound // 2
    # Generate Dataset
    randomMultipliers = [];results = [];noiseValues = []
    for i in range(equationsToGenerate):
        r = randint(1, multiplierBound);s = randint(0, errorBound)
        c = (r*secretKey + s) % q
        c = (c - halfErrorBound) % q
        #print(i, " : ", math.log(abs(s - halfErrorBound), 2))
        randomMultipliers.append(r);results.append(c);noiseValues.append(s)

    #Predict
    prediction = HNP_EliminateSecretKey_Projection(q, randomMultipliers, results, errorBound//2,noOfLeakedBits)
    print(f"Target: {secretKey}")
    print(f"Prediction: {prediction}: {math.log(prediction,2)}")

if __name__ == "__main__":
  random.seed(96650946)
  randomValue = (randint(2**134, 2**135))
  secretKey = randomValue
  TestHNP_EliminateSecretKey(secretKey)

24318916724595889543256022920101731919724 : 134.15920276149387
182138806401864986630747728019500395017279267722148757875157906534900605 : 236.72193307194215
161609860576855866324778783330323537954202490841459610659240277587201096 : 236.54940996338036
93684710802865729252318399048997917993413956273062883348904197099726075 : 235.76278026365327
133262445996763816134526272315090399179754346230081072747505894131718116 : 236.27116501624832
170746473408827405517805179830244229696747019369931675604676715984343216 : 236.62875051845728
114840030616264332070073146133605059877417103976362634365389733855248042 : 236.0565203572819
95074040201583024082011470465289272025015438068468682488517888546392935 : 235.78401811164406
54025433239984980021731389450851882949604559795634610008413396740138812 : 234.96860537848607
134384235848957351573192486892595614678780525613803797444751809726161366 : 236.28325864746745
157660858321959739136961672769692351431287122105828413530187194858524261 : 236.5137192708539
85

#Volume Shortcuts
The volume permits you set the random multiplier to 1
But you need errorMaxExponent to be low like 150 bits low

In [9]:
def TestHNP_GuessingBits_Everything1(secretKey, noOfGuessedBits):
    _ = FPLLL.set_precision(1000)
    q = 205115282021455665897114700593932402728804164701536103180137503955397371
    equationsToGenerate = 44
    errorMaxExponent = 140
    errorBound = 2**errorMaxExponent
    noOfLeakedBits = math.ceil(math.log(q, 2)) - errorMaxExponent
    halfErrorBound = errorBound // 2
    # Generate Dataset
    randomMultipliers = [];results = [];noiseValues = []
    for i in range(equationsToGenerate):
        r = 1
        s = randint(0, errorBound)
        c = (r*secretKey + s) % q
        c = (c - halfErrorBound) % q
        randomMultipliers.append(r);results.append(c);noiseValues.append(s)
    print(f"Secret key: {secretKey}")
    HNP_GuessingBits(q, randomMultipliers, results, errorBound//2,noOfLeakedBits, noOfGuessedBits)

if __name__ == "__main__":
  #random.seed(96650956)
  randomValue = (randint(2**134, 2**135))
  noOfGuessedBits = 102
  secretKey = randomValue
  TestHNP_GuessingBits_Everything1(secretKey, noOfGuessedBits)


Secret key: 38137865720659829566245308234913299801288
alpha0: 5070602400912917605986812821504, 102.00000000000001
alpha0: 108493843415933484231402189209035790614528, 136.31666506830226


#Double Volume Trick
We can use both volumes, si max exponent can be higher



In [10]:
def HNP_GuessingBits_DoubleVolume(prime, multipliers, observations, errorBound, noOfLeakedBits, noOfGuessedBits, volume):
  d = len(multipliers)
  embeddingFactor = prime
  bkzBlockSize = 20
  param = BKZ.Param(block_size=bkzBlockSize, flags=BKZ.AUTO_ABORT)
  for lsb in range(0, 2 ** noOfGuessedBits):
    lattice = HNP_BuildLattice_GuessingBits(prime, multipliers, observations, noOfLeakedBits, noOfGuessedBits, lsb,embeddingFactor)
    #print(lattice)
    LLL.reduction(lattice)
    for i in range(d+2):
      temp = abs(lattice[i, d])
      alpha0 = (((temp >> noOfGuessedBits) << noOfGuessedBits) + lsb) >> volume
      alpha1 = (prime - ((temp >> noOfGuessedBits) << noOfGuessedBits)+lsb) >> volume
      logAlpha0 = 0; logAlpha1=0;
      if(alpha0 > 0):
        logAlpha0 = math.log(alpha0,2)
        if(logAlpha0 > 133 and logAlpha0 < 135):
          print(f"alpha0: {alpha0}, {logAlpha0}")
      if(alpha1 > 0):
        logAlpha1 = math.log(alpha1,2)
        if(logAlpha1 > 133 and logAlpha1 < 135):
          print(f"alpha1: {alpha1}, {logAlpha1}")
    break

    print(f"Done lsb:{lsb}\n")


def TestHNP_GuessingBits_DoubleVolume(secretKey, noOfGuessedBits, volume):
    _ = FPLLL.set_precision(1000)
    q = 205115282021455665897114700593932402728804164701536103180137503955397371
    equationsToGenerate = 200
    errorMaxExponent = 234
    errorBound = 2**errorMaxExponent
    noOfLeakedBits = 4#math.ceil(math.log(q, 2)) - errorMaxExponent
    halfErrorBound = errorBound // 2
    multiplierMaxExponent = 2
    multiplierBound= 2**multiplierMaxExponent
    # Generate Dataset
    randomMultipliers = [];results = [];noiseValues = []
    for i in range(equationsToGenerate):
        r = -1 * randint(1, 3);
        s = randint(errorBound // 4, errorBound)
        c = (r*secretKey + s) % q
        c = (c - halfErrorBound) % q
        randomMultipliers.append(r);results.append(c);noiseValues.append(s)
    print(f"\nSecret key: {secretKey >> volume}, {math.log(secretKey >> volume,2)}")
    HNP_GuessingBits_DoubleVolume(q, randomMultipliers, results, errorBound//2,noOfLeakedBits, noOfGuessedBits,volume)

if __name__ == "__main__":
  #random.seed(6650956)
  volume = 97
  randomValue = 26921994609240420786291109313825160459954 #(randint(2**134, 2**135))
  noOfGuessedBits = 102
  secretKey0 = randomValue << volume
  secretKey1 = (randomValue + 1) << volume
  TestHNP_GuessingBits_DoubleVolume(secretKey0, noOfGuessedBits, volume)
  TestHNP_GuessingBits_DoubleVolume(secretKey1, noOfGuessedBits, volume)



Secret key: 26921994609240420786291109313825160459954, 134.30590909646273
alpha0: 16158425758916293606838476733221882079296, 133.56941044528998

Secret key: 26921994609240420786291109313825160459955, 134.30590909646273
alpha0: 16435972410323303127244339473958624962592, 133.5939806093846


In [11]:
def HNP_GuessingBits_DoubleVolume(prime, multipliers, observations, errorBound, noOfLeakedBits, noOfGuessedBits, volume):
  d = len(multipliers)
  embeddingFactor = prime
  bkzBlockSize = 20
  param = BKZ.Param(block_size=bkzBlockSize, flags=BKZ.AUTO_ABORT)
  for lsb in range(0, 2 ** noOfGuessedBits):
    lattice = HNP_BuildLattice_GuessingBits(prime, multipliers, observations, noOfLeakedBits, noOfGuessedBits, lsb,embeddingFactor)
    #print(lattice)
    LLL.reduction(lattice)
    for i in range(d+2):
      temp = abs(lattice[i, d])
      alpha0 = (((temp >> noOfGuessedBits) << noOfGuessedBits) + lsb) >> volume
      alpha1 = (prime - ((temp >> noOfGuessedBits) << noOfGuessedBits)+lsb) >> volume
      logAlpha0 = 0; logAlpha1=0;
      if(alpha0 > 0):
        logAlpha0 = math.log(alpha0,2)
        if(logAlpha0 > 133 and logAlpha0 < 135):
          print(f"alpha0: {alpha0}, {logAlpha0}")
      if(alpha1 > 0):
        logAlpha1 = math.log(alpha1,2)
        if(logAlpha1 > 133 and logAlpha1 < 135):
          print(f"alpha1: {alpha1}, {logAlpha1}")
    break

    print(f"Done lsb:{lsb}\n")


def TestHNP_GuessingBits_DoubleVolume(secretKey, noOfGuessedBits, volume):
    _ = FPLLL.set_precision(1000)
    q = 205115282021455665897114700593932402728804164701536103180137503955397371
    equationsToGenerate = 200
    errorMaxExponent = 232
    errorBound = 2**errorMaxExponent
    noOfLeakedBits = math.ceil(math.log(q, 2)) - errorMaxExponent
    halfErrorBound = errorBound // 2
    multiplierMaxExponent = 2
    multiplierBound= 2**multiplierMaxExponent
    # Generate Dataset
    randomMultipliers = [];results = [];noiseValues = []
    for i in range(equationsToGenerate):
        r = -1 #* randint(1, 3);
        s = randint(errorBound//2, errorBound)
        c = (r*secretKey + s) % q
        c = (c - halfErrorBound) % q
        randomMultipliers.append(r);results.append(c);noiseValues.append(s)
    print(f"\nSecret key: {secretKey >> volume}, {math.log(secretKey >> volume,2)}")
    HNP_GuessingBits_DoubleVolume(q, randomMultipliers, results, errorBound//2,noOfLeakedBits, noOfGuessedBits,volume)

if __name__ == "__main__":
  #random.seed(6650956)
  volume = 97
  randomValue = (randint(2**134, 2**135))
  noOfGuessedBits = 102
  secretKey0 = randomValue << volume
  secretKey1 = (randomValue + 1) << volume
  TestHNP_GuessingBits_DoubleVolume(secretKey0, noOfGuessedBits, volume)
  TestHNP_GuessingBits_DoubleVolume(secretKey1, noOfGuessedBits, volume)



Secret key: 36968891091988829127827602357037879754738, 134.7634355650115
alpha0: 26055974810530674886329102566570365044896, 134.25873802562418
alpha0: 19583685629269958562414025874976723696800, 133.846776099695
alpha0: 32528263991791391210244179258164006392992, 134.5788176244132
alpha0: 32528263991791391210244179258164006392992, 134.5788176244132
alpha0: 19583685629269958562414025874976723696800, 133.846776099695
alpha0: 19583685629269958562414025874976723696800, 133.846776099695
alpha0: 32528263991791391210244179258164006392992, 134.5788176244132
alpha0: 32528263991791391210244179258164006392992, 134.5788176244132
alpha0: 19583685629269958562414025874976723696800, 133.846776099695
alpha0: 19583685629269958562414025874976723696800, 133.846776099695
alpha0: 19583685629269958562414025874976723696800, 133.846776099695
alpha0: 32528263991791391210244179258164006392992, 134.5788176244132
alpha0: 19583685629269958562414025874976723696800, 133.846776099695
alpha0: 195836856292699585624140258